<a href="https://colab.research.google.com/github/smartgrids-aau/GreenCodesPython/blob/main/Exemple_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Example 3.1
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d


def PV_model(Va, Suns, TaC):

    Isc = 5
    Voc = 36
    T_ref = 25
    Ki = 0.005
    Iph = Suns * (Isc + Ki * (TaC - T_ref))
    Io = 1e-10
    n = 1.3
    Vt = 0.026
    return Iph - Io * (np.exp(Va / (n * Vt)) - 1)


TaC = 25            # Cell temperature (°C)
C = 0.5             # Step size
Suns_val = 0.028    # (1 G = 1000 W/m^2)
Va = 31             # PV voltage

Ia = PV_model(Va, Suns_val, TaC)
Pa = Ia * Va        # PV output power
Vref_new = Va + C   # New reference voltage

Va_array = []
Pa_array = []

Suns_data = np.array([
    [0, 0.1], [1, 0.2], [2, 0.3], [3, 0.3], [4, 0.5],
    [5, 0.6], [6, 0.7], [7, 0.8], [8, 0.9], [9, 1.0],
    [10, 1.1], [11, 1.2], [12, 1.3], [13, 1.4]
])

x = Suns_data[:, 0]
y = Suns_data[:, 1]
xi = np.arange(1, 201)
interp_func = interp1d(x, y, kind='cubic', fill_value='extrapolate')
yi = interp_func(xi)

for i in range(14):
    Suns = yi[i]
    Va_new = Vref_new
    Ia_new = PV_model(Va, Suns, TaC)
    Pa_new = Va_new * Ia_new
    deltaPa = Pa_new - Pa

    if deltaPa > 0:
        if Va_new > Va:
            Vref_new = Va_new + C
        else:
            Vref_new = Va_new - C
    elif deltaPa < 0:
        if Va_new > Va:
            Vref_new = Va_new - C
        else:
            Vref_new = Va_new + C
    else:
        V_ref = Va_new

    Va = Va_new
    Pa = Pa_new

    Va_array.append(Va)
    Pa_array.append(Pa)


/tmp/ipython-input-6-1856769395.py:17: RuntimeWarning: overflow encountered in exp
  return Iph - Io * (np.exp(Va / (n * Vt)) - 1)
/tmp/ipython-input-6-1856769395.py:49: RuntimeWarning: invalid value encountered in scalar subtract
  deltaPa = Pa_new - Pa
